## ==> PIPELINE QUI CONSTRUIT F1 et F2, les deux bases de données des stats historiques pour la ligue1 et la ligue2, mise à jour jusqu'aux matches recement joués pendant la season en cours
### ==> à lancer à la date d'après de chaque journée <==

### IMPORTER les Libraries nécessaires au fonctionnement

In [ ]:
import os
import datetime
import urllib.request
import subprocess
from google.cloud import bigquery

## Panneau des parametres à regler


In [ ]:
# l'année où le scrapping à va commencer
yr_dbt = 19

# l'année où le scrpping va se terminer (année en cours) le +1 est pour prendre en compte le fin de range()
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
yr_fin = int(date.strftime("%y"))+1 

# les chemins des dossiers dans le bucket
path_bucket_lake = "gs://bucket-git-m2i/DB/historique/"
path_bucket_gold = "gs://bucket-git-m2i/DB/golden/" 

# table_id to the ID of the table to create.
table_id = "micro-atrium-360309.m2i_pronostics.merging-F"

# pour execution schedulée, extraction de l'ID du projet temporaire créé
project_number = os.environ["CLOUD_ML_PROJECT_ID"] #si execution schedulée


## Scrapper les données des stats historiques et les stocker dans une destination locale
1. verifier si le dossier existe sinon le creer temporairement
2. telecharger les fichiers data en question dans le dossier / pour chaque ligue
3. creation un seul fichier data pour toutes les données scrappées / pour chaque ligue
4. Deplacer le BDD final (le fichier F1 avec les fichiers telechargés) à notre bucket dans le chemin destiné

In [ ]:
for ligue in range(1,3):
    # 1.
    s_dir=f"./data_ligue_{ligue}/"
    if not os.path.exists(s_dir):
        os.mkdir(s_dir)
    # 2.
    for i in range(yr_dbt,yr_fin):
        urllib.request.urlretrieve(f"https://www.football-data.co.uk/mmz4281/{i}{i+1}/F{ligue}.csv", f"{s_dir}F{ligue}_{i}_{i+1}.csv")
    # 3.
    merged =open(f"{s_dir}F{ligue}.csv","w")    
    merged.write('Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA\n')  
    for yr in range(yr_dbt,yr_fin):
        f = open(f"{s_dir}F{ligue}_{yr}_{yr+1}.csv")
        next(f)
        for ligne in f.readlines():
            merged.write(ligne)
        f.close() # not really needed
    merged.close()
    
    # 4.
    path_local = f"{s_dir}F{ligue}.csv"
    subprocess.call(f"gsutil cp {path_local} {path_bucket_lake}F{ligue}.csv", shell=True)
    for i in range(yr_dbt,yr_fin):
        subprocess.call(f"gsutil cp ./{s_dir}F{ligue}_{i}_{i+1}.csv {path_bucket_lake}", shell=True)


## Merging des csv contenant les résultats historiques des 5 dernières saisons -> csv comprenant home, away, score_home, score_away

In [ ]:
# creer un path local pour le fichier où on va merger les F1 et F2
path_local = f"./data_ligue_1/merging_F.csv"

# on ouvre le fichier
merged_F =open(path_local,"a")

# on commence la construction par la tête des titres de columns
merged_F.write('Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA\n')  

# en boucle for on ouvre F1 et F2 et on copie chaque ligne sauf la tête
for ligue in range(1,3):
    # ouvrir le fichier F{ligue}=F1 et F2
    f = open(f"./data_ligue_{ligue}/F{ligue}.csv")
    # on evite premier ligne
    next(f)
    # dans le fichier F ouvert on va copier (write) chaque ligne el l'envoyer vers merged_F
    for ligne in f.readlines():
        merged_F.write(ligne)
    f.close() # not really needed
merged_F.close()

# copier le fichier final merging_F vers le bucket dans le dossier golden
subprocess.call(f"gsutil cp {path_local} {path_bucket_gold}merging_F.csv", shell=True)


## Liberer l'espace local par supprimer le dossier creer temporairement

In [ ]:
for ligue in range (1,3):
    s_dir = f"./data_ligue_{ligue}/"
    !rm -r $s_dir


### Le csv créé (merging_F) est importé dans un dataset bigQuery (en écrasant les donnnées)

In [ ]:
# Construct a BigQuery client object.
#client = bigquery.Client() #si execution manuelle
client = bigquery.Client(project_number) #si execution schedulée

job_config = bigquery.LoadJobConfig(
    skip_leading_rows=1,
    write_disposition="WRITE_TRUNCATE",
    autodetect=True,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
uri = f"{path_bucket_gold}merging_F.csv"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))